In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv")


In [ ]:
df

In [ ]:
len(df["City (Billing)"].unique())

In [ ]:
df["City (Billing)"] = df["City (Billing)"].str.lower()
df["City (Billing)"] = df["City (Billing)"].str.strip()
len(df["City (Billing)"].unique())

In [ ]:
cities = set()

In [ ]:
pk_cities = pd.read_csv("../input/pakistan-cities/pk.csv")
cities.update(pk_cities['city'].str.lower())
len(cities)

In [ ]:
pk_cities = pd.read_csv("../input/pakistani-cities-by-wiki/cities.csv")
pk_cities.columns = ["city","Type","District"]
cities.update(pk_cities["city"].str.lower().tolist())
len(cities)

In [ ]:
pk_cities = pd.read_csv("../input/pakistan-population-census-2017-at-village-level/Pakistan_Population_2017_Census_Village_Administration_Level.csv")
cities.update(pk_cities["Village"].str.lower().tolist())
len(cities)

In [ ]:
def get_proper_city_name(_city):
    if str(_city) in cities:
        return _city
    for part in str(_city).split():
        if str(part) in cities:
            return part
    return "NO CHANGE" 

In [ ]:
df['city'] = df['City (Billing)'].apply(get_proper_city_name)

In [ ]:
no_change = df[df["city"] == "NO CHANGE"]
len(no_change)

In [ ]:
no_change_map = dict()
for index, row in no_change.iterrows():
    val = no_change_map.get(row['City (Billing)'])
    if( val == None ):
        no_change_map[row['City (Billing)']] = 1
    else:
        no_change_map[row['City (Billing)']] = val + 1

In [ ]:
count = 1
for k in no_change_map.keys():
    if(no_change_map[k] >= 6  and '??' not in k):
        print(str(count) , "\t" , no_change_map[k],"\t", k)
        count+=1

In [ ]:
missing_cities = ['fateh jang','chakdara','rawalakot','talagang','haroonabad','buner','battagram','taxila','shahkot','depalpur','skardu',
                 'mingora','hazro','fort abbas','daharki','usta muhammad','bhimber','timergara','nankana sahib','kashmore','tando adam','dina',
                 'pindigheb','sakrand','shahdadkot','murree','besham','poonch']
other_counteries = ['abu dhabi','london','dubai']
replace_name = {'mirpurkhas': 'mirpur khas', 'wahcantt':'wah cantt' , 'fsd':'faisalabad',
                'ahmed pur east':'ahmedpur east', 'rwp':'rawalpindi','lhr':'lahore','bahawal pur':'bahawalpur', 'rawlakot':'rawalakot',
               'khi':'karachi','peshawer':'peshawar','pirmahal':'pir mahal','sadiqbad':'sadiqabad','faislabad':'faisalabad' }

In [ ]:
cities.update(missing_cities)

In [ ]:
for old_value in replace_name.keys():
    df['City (Billing)'] = df['City (Billing)'].replace([old_value],replace_name[old_value])

In [ ]:
df['city'] = df['City (Billing)'].apply(get_proper_city_name)

In [ ]:
no_change = df[df["city"] == "NO CHANGE"]
len(no_change)

In [ ]:
len(df["city"].unique())

In [ ]:
_map = dict()
for index, row in df.iterrows():
    val = _map.get(row['city'])
    if( val == None ):
        _map[row['city']] = 1
    else:
        _map[row['city']] = val + 1

In [ ]:
# Wants to draw Geographical representation of Order, but it will take time. Next will focus on Questions

In [ ]:
row_has_NaN = df.isnull().any(axis=1)
df[row_has_NaN]

In [ ]:
df = df.dropna()

In [ ]:
final_map = dict()        
for k,v in _map.items(): 
    if ( v > 75 and k != 'NO CHANGE' ):
        final_map[k+"\n"+str(v)] = v

In [ ]:
import squarify
import matplotlib.pyplot as plt

plt.figure(figsize = (25, 10))
squarify.plot(sizes = final_map.values(), alpha = 0.8,
              label = final_map.keys(), text_kwargs={'fontsize':14})

plt.title('Cities with more than 75 orders ', fontsize = 30)
plt.axis('off')
plt.show()

In [ ]:
# Top selling book
df["Book Name"].value_counts()

In [ ]:
books = dict()
for row in df.itertuples():
    for book in str(row[3]).replace(',','/').split('/'):
        val = books.get(book)
        if(val == None):
            books[book] = 1
        else:
            books[book] = val +1

In [ ]:
books = {k: v for k, v in sorted(books.items(),reverse=True, key=lambda item: item[1])} 
books = {k: v for k, v in books.items() if v > 300} 
books

In [ ]:
import seaborn as sns
plt.figure(figsize = (25, 10))

sns.barplot(x = [k for k, v in books.items()] , y = [v for k, v in books.items()], color = "Blue", label = "count")


plt.xlabel("Books", fontsize = 30)
plt.ylabel("Sale", fontsize = 30)
plt.title("Best Selling Books", fontsize = 30)
plt.xticks(rotation = 60)
plt.legend()
plt.show()

In [ ]:
print (df['Order Date'].dtypes)

In [ ]:
df['Order Date']= pd.to_datetime(df['Order Date'])

In [ ]:
df.dtypes

In [ ]:
dd={}

for name, group in df['Order Date'].groupby([pd.DatetimeIndex(df['Order Date']).year,pd.DatetimeIndex(df['Order Date']).month]):
    dd[name] = len(group)
print(dd)

In [ ]:
dd.values()

In [ ]:
dd = {"2019-10":0,"2019-11":0,"2019-12":0,"2020-1":0,"2020-2":0,"2020-3":0,"2020-4":0,"2020-5":0,"2020-6":0,"2020-7":0,"2020-8":0,"2020-9":0,"2020-10":0,"2020-11":0,"2020-12":0,"2021-1":0}

for name, group in df['Order Date'].groupby([pd.DatetimeIndex(df['Order Date']).year,pd.DatetimeIndex(df['Order Date']).month]):
    dd[str(name).replace(', ','-').replace('(','').replace(')','')] = len(group)

print(dd)

In [ ]:
plt.figure(figsize = (25, 10))
d = {'years': dd.keys(), 'sale' : dd.values()}
sns.lineplot(data=pd.DataFrame(data=d), x="years", y="sale")
plt.title('Per month Sale ', fontsize = 30)
plt.show()

In [ ]:
returnedDF = df[df['Order Status']=='Returned']
returnedDF

In [ ]:
# Find a correlation between date and time with order status
dd = {"2019-10":0,"2019-11":0,"2019-12":0,"2020-1":0,"2020-2":0,"2020-3":0,"2020-4":0,"2020-5":0,"2020-6":0,"2020-7":0,"2020-8":0,"2020-9":0,"2020-10":0,"2020-11":0,"2020-12":0,"2021-1":0}

for name, group in returnedDF['Order Date'].groupby([pd.DatetimeIndex(returnedDF['Order Date']).year,pd.DatetimeIndex(returnedDF['Order Date']).month]):
    dd[str(name).replace(', ','-').replace('(','').replace(')','')] = len(group)
print(dd)

In [ ]:
plt.figure(figsize = (25, 10))
d = {'years': dd.keys(), 'sale' : dd.values()}
sns.lineplot(data=pd.DataFrame(data=d), x="years", y="sale")
plt.title('Order retuen per month ', fontsize = 30)
plt.show()

In [ ]:
dd = {"2019-10":0,"2019-11":0,"2019-12":0,"2020-1":0,"2020-2":0,"2020-3":0,"2020-4":0,"2020-5":0,"2020-6":0,"2020-7":0,"2020-8":0,"2020-9":0,"2020-10":0,"2020-11":0,"2020-12":0,"2021-1":0}

completedDF = df[df['Order Status']=='Completed']
for name, group in completedDF['Order Date'].groupby([pd.DatetimeIndex(completedDF['Order Date']).year,
                                                      pd.DatetimeIndex(completedDF['Order Date']).month]):
    dd[str(name).replace(', ','-').replace('(','').replace(')','')] = len(group)
print(dd)

In [ ]:
plt.figure(figsize = (25, 10))
d = {'years': dd.keys(), 'order' : dd.values()}
pd.DataFrame(data=d)

sns.lineplot(data=pd.DataFrame(data=d), x="years", y="order")
plt.title('Complete order per month ', fontsize = 30)
plt.show()

In [ ]:
# Find a correlation between city and order status
dd={}

for name, group in returnedDF.groupby(['city']):
    #print(name, "\t", len(group))
    dd[name] = len(group)
sorted(dd)
dd = {k: v for k, v in sorted(dd.items(),reverse=True, key=lambda item: item[1])} 
dd = {k: v for k, v in dd.items() if v > 9} 
dd
# Mostly return orders are from unknown area, which means that we should verify address before processing

In [ ]:
# Find a correlation between city and order status
dd={}

for name, group in completedDF.groupby(['city']):
    #print(name, "\t", len(group))
    dd[name] = len(group)
sorted(dd)
dd = {k: v for k, v in sorted(dd.items(),reverse=True, key=lambda item: item[1])} 
dd = {k: v for k, v in dd.items() if v > 100} 
dd

In [ ]:
# Can we predict number of orders, or book names in advance?
top_20_books = ['انٹرنیٹ سے پیسہ کمائیں',
 'Python Programming',
 'Artificial Intelligence',
 'Blockchain',
 'Cryptocurrency And Bitcoin',
 'Product Management',
 'Justju ka safar-1',
 'ڈیٹا سائنس',
 'مشین لرننگ',
 'Sukkur To Florida',
 '(C++)',
 'R ka Taaruf',
 'Introduction to Blockchain with Case Studies',
 'Molo Masali',
 'ڈیٹا سائنس ۔ ایک تعارف',
 'Lucky Draw - Free Book',
 'Kaggle for Begginers',
 'ایک تھا الگورتھم',
 'Column Nigari Kese Sekhain',
 'Linux - An Introduction']

top_20_books[1:10]

In [ ]:
# Find any hidden patterns that are counter-intuitive for a layman
## more than 50% orders are from top 20 Books 
top_20_books_orders_df = df[df['Book Name'].isin(top_20_books)]
len(top_20_books_orders_df)

In [ ]:
# Find any hidden patterns that are counter-intuitive for a layman
## more than 25% orders are from top 10 Books --
top_10_books_orders_df = df[df['Book Name'].isin(top_20_books[1:10])]
top_10_books_orders_df

In [ ]:
x = ["2019-10","2019-11","2019-12","2020-1","2020-2","2020-3","2020-4","2020-5","2020-6","2020-7","2020-8","2020-9","2020-10","2020-11","2020-12","2021-1"]
y = {"2019-10":0,"2019-11":0,"2019-12":0,"2020-1":0,"2020-2":0,"2020-3":0,"2020-4":0,"2020-5":0,"2020-6":0,"2020-7":0,"2020-8":0,"2020-9":0,"2020-10":0,"2020-11":0,"2020-12":0,"2021-1":0}
z = {"2019-10":0,"2019-11":0,"2019-12":0,"2020-1":0,"2020-2":0,"2020-3":0,"2020-4":0,"2020-5":0,"2020-6":0,"2020-7":0,"2020-8":0,"2020-9":0,"2020-10":0,"2020-11":0,"2020-12":0,"2021-1":0}

In [ ]:
#Can we predict number of orders, or book names in advance?
## Gufhtugu publisher will get more Orders of Top 20 books 
dic_list = []
for i in range(1,16):

    book_df = df[df['Book Name']== top_20_books[i]]
    y = z
    for name, group in book_df['Order Date'].groupby([pd.DatetimeIndex(book_df['Order Date']).year,
                                                      pd.DatetimeIndex(book_df['Order Date']).month]):
        y[str(name).replace(', ','-').replace('(','').replace(')','')] = len(group)
    dic_list.append(y.copy())
print(len(dic_list))

In [ ]:
books_df = pd.DataFrame(columns=['year-month','book','sale_count'])
count = 0
for index,l in enumerate(dic_list):
    print(top_20_books[index])
    #d = {'year-month': x, 'book' : top_20_books[index], 'sale_count': y.values()}
    for k,v in l.items():
        books_df = books_df.append({'year-month': k, 'book' : top_20_books[index], 'sale_count': v}, ignore_index=True)
    count +=1

In [ ]:
a4_dims = (11.7, 8.27)

grid = sns.FacetGrid(books_df, col="book", hue="book", palette="tab20c",
                     col_wrap=4, height=4.5)

grid.map(plt.axhline, y=0, ls=":", c=".7")
grid.map(plt.plot, "year-month", "sale_count", marker="o")

grid.set(xticks=np.arange(16), yticks=[0, 500],
         xlim=(0, 16), ylim=(0, 500))

grid.set_xticklabels(rotation=60)

grid.fig.tight_layout(w_pad=4)